<a href="https://colab.research.google.com/github/Yasiyass/A-BERT-Transformer-COVID19-Q-A-System/blob/main/experimental_pocs/15_using_speaker_diarization_with_assemblyai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Speaker Diarization with AssemblyAI

> 📚 This cookbook has an accompanying article with a complete walkthrough "[Level up Your RAG Application with Speaker Diarization](https://haystack.deepset.ai/blog/level-up-rag-with-speaker-diarization)"

LLMs excel with text data, answering complex questions without manual reading or searching. When dealing with audio or video, providing transcription is key. Transcription captures spoken content of the audio or video, but in multi-speaker recordings, it may miss non-verbal information and fail to convey speaker count or individual remarks. Therefore, to maximize the LLM's potential with such recordings, **Speaker Diarization** is essential!

In this example, we'll build a RAG application with speaker labels for audio files. This application will use [Haystack](https://github.com/deepset-ai/haystack) and speaker diarization models by [AssemblyAI](https://www.assemblyai.com/).

📚 Useful Sources:
* [Integration: AssemblyAI](https://haystack.deepset.ai/integrations/assemblyai)

>[Level up Your RAG Application with Speaker Diarization](#scrollTo=vcxHd6VB31a1)

>>[Install the Dependencies](#scrollTo=FIJ2fJbR696M)

>>[Download The Audio Files](#scrollTo=XnTLMNd-UxqR)

>>[Add Your API Keys](#scrollTo=GQ78gO-1VhGL)

>>[Index Speaker Labels to Your DocumentStore](#scrollTo=JswXLocxyQX6)

>>[RAG Pipeline with Speaker Labels](#scrollTo=FSWZNQBhyGJI)

>>[Test RAG with Speaker Labels](#scrollTo=hVRO6z5xx6mg)



## Install the Dependencies

In [1]:
%%bash
pip install haystack
pip install assemblyai-haystack
pip install "sentence-transformers>=2.2.0"
pip install "huggingface_hub>=0.22.0"
pip install --upgrade gdown

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.6/179.6 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 688.1/688.1 kB 22.0 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.8/71.8 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.8/104.8 kB 12.5 MB/s eta 0:00:00
  Created wheel for construct: filename=construct-2.5.3-py2.py3-none-any.whl size=71822 sha256=6dd3f29a9bcf74de69b09dd0ff23d8effbc78589acdaa15b759b6d25d1849249
  Stored in directory: /root/.cache/pip/wheels/ce/16/f9/f48a4c1a687e0848495c1a95dd9a87a246974b9318240d140b
Successfully built construct
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.7/266.7 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.3/68.3 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

## Download The Audio Files

We extracted the audio from youtube videos and saved them in a Google Drive Folder for you: https://drive.google.com/drive/folders/10zsFuHmj3oytYMyGrLdytpW-6JzT9T_W?usp=drive_link

You can run the code below to download the audio files to this colab notebook under "Files" tab on the left bar.

In [2]:
!gdown https://drive.google.com/drive/folders/10zsFuHmj3oytYMyGrLdytpW-6JzT9T_W -O "/content" --folder

Retrieving folder contents
Processing file 12654ySXSYc2rZnPgNxXZwWt2hH-kTNDZ Netflix_Q4_2023_Earnings_Interview.mp3
Processing file 1Zb15D_nrBzWlM3K8FuPOmyiCiYvsuJLD Panel_Discussion.mp3
Processing file 1FFKGEZAUSmJayZgGaAe1uFP9HUtOK5m- Working_From_Home_Debate.mp3
Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=12654ySXSYc2rZnPgNxXZwWt2hH-kTNDZ
To: /content/Netflix_Q4_2023_Earnings_Interview.mp3
100% 39.1M/39.1M [00:00<00:00, 62.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Zb15D_nrBzWlM3K8FuPOmyiCiYvsuJLD
To: /content/Panel_Discussion.mp3
100% 21.8M/21.8M [00:00<00:00, 53.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=1FFKGEZAUSmJayZgGaAe1uFP9HUtOK5m-
To: /content/Working_From_Home_Debate.mp3
100% 4.45M/4.45M [00:00<00:00, 168MB/s]
Download completed


## Add Your API Keys

Enter the API keys from [AssemblyAI](https://www.assemblyai.com/) and [Hugging Face](https://huggingface.co/settings/tokens):

In [3]:
import os
from getpass import getpass

ASSEMBLYAI_API_KEY = getpass("Enter your ASSEMBLYAI_API_KEY: ")
os.environ["HF_API_TOKEN"] = getpass("HF_API_TOKEN: ")

Enter your ASSEMBLYAI_API_KEY: ··········
HF_API_TOKEN: ··········


## Index Speaker Labels to Your DocumentStore

Build a pipeline to generate speaker labels and index them into a DocumentStore with their embeddings. In this pipeline, you need:

* [InMemoryDocumentStore](https://docs.haystack.deepset.ai/docs/inmemorydocumentstore): to store your documents without external dependencies or extra setup
* [AssemblyAITranscriber](https://github.com/AssemblyAI/assemblyai-haystack/blob/main/assemblyai_haystack/transcriber.py): to create speaker_labels for the given audio file and convert them into Haystack Documents
* [DocumentSplitter](https://docs.haystack.deepset.ai/docs/documentsplitter): to split your documents into smaller chunks
* [SentenceTransformersDocumentEmbedder](https://docs.haystack.deepset.ai/docs/sentencetransformersdocumentembedder): to create embeddings for each document using sentence-transformers models
* [DocumentWriter](https://docs.haystack.deepset.ai/docs/documentwriter): to write these documents into your document store

Note: The speaker information will be saved in the `meta` of the Document object

In [4]:
from haystack.components.writers import DocumentWriter
from haystack.components.preprocessors import DocumentSplitter
from haystack.components.embedders import SentenceTransformersDocumentEmbedder
from haystack import Pipeline
from haystack.document_stores.in_memory import InMemoryDocumentStore
from assemblyai_haystack.transcriber import AssemblyAITranscriber
from haystack.document_stores.types import DuplicatePolicy
from haystack.utils import ComponentDevice

speaker_document_store = InMemoryDocumentStore()
transcriber = AssemblyAITranscriber(api_key=ASSEMBLYAI_API_KEY)
speaker_splitter = DocumentSplitter(
    split_by = "sentence",
    split_length = 10,
    split_overlap = 1
)
speaker_embedder = SentenceTransformersDocumentEmbedder(device=ComponentDevice.from_str("cuda:0"))
speaker_writer = DocumentWriter(speaker_document_store, policy=DuplicatePolicy.SKIP)

indexing_pipeline = Pipeline()
indexing_pipeline.add_component(instance=transcriber, name="transcriber")
indexing_pipeline.add_component(instance=speaker_splitter, name="speaker_splitter")
indexing_pipeline.add_component(instance=speaker_embedder, name="speaker_embedder")
indexing_pipeline.add_component(instance=speaker_writer, name="speaker_writer")

indexing_pipeline.connect("transcriber.speaker_labels", "speaker_splitter")
indexing_pipeline.connect("speaker_splitter", "speaker_embedder")
indexing_pipeline.connect("speaker_embedder", "speaker_writer")

🚅 Components
  - transcriber: AssemblyAITranscriber
  - speaker_splitter: DocumentSplitter
  - speaker_embedder: SentenceTransformersDocumentEmbedder
  - speaker_writer: DocumentWriter
🛤️ Connections
  - transcriber.speaker_labels -> speaker_splitter.documents (List[Document])
  - speaker_splitter.documents -> speaker_embedder.documents (List[Document])
  - speaker_embedder.documents -> speaker_writer.documents (List[Document])

Give an `audio_file_path` and run your pipeline

In [5]:
audio_file_path = "/content/Panel_Discussion.mp3" #@param ["/content/Netflix_Q4_2023_Earnings_Interview.mp3", "/content/Working_From_Home_Debate.mp3", "/content/Panel_Discussion.mp3"]
indexing_pipeline.run(
    {
        "transcriber": {
            "file_path": audio_file_path,
            "summarization": None,
            "speaker_labels": True
        },
    }
)

/usr/local/lib/python3.10/dist-packages/sentence_transformers/SentenceTransformer.py:99: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v3 of SentenceTransformers.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

{'transcriber': {'transcription': [Document(id=c03d451fa6cbcb57d293b91b98b9b932c4bed6167d9166aed7d733cfd8c1ee2c, content: 'I want to start with you, Amy, because I know you, obviously at Shell have had AI as part of the wor...', meta: {'transcript_id': '67987274-f7a6-43d1-b959-4770456535d7', 'audio_url': 'https://cdn.assemblyai.com/upload/5d9ae4b8-4b30-446d-a1cf-1d2bb9fd32d7'})]},
 'speaker_writer': {'documents_written': 62}}

## RAG Pipeline with Speaker Labels

Build a RAG pipeline to generate answers to questions about the recording. Ensure that speaker information (provided through the metadata of the document) is included in the prompt for the LLM to distinguish who said what. For this pipeline, you need:

* [SentenceTransformersTextEmbedder](https://docs.haystack.deepset.ai/docs/sentencetransformerstextembedder): To create an embedding for the user query using sentence-transformers models
* [InMemoryEmbeddingRetriever](https://docs.haystack.deepset.ai/docs/inmemoryembeddingretriever): to retrieve `top_k` relevant documents to the user query
* [PromptBuilder](https://docs.haystack.deepset.ai/docs/promptbuilder): to provide a RAG prompt template with instructions to be filled with retrieved documents and the user query
* [HuggingFaceTGIGenerator](https://docs.haystack.deepset.ai/docs/huggingfacetgigenerator): to infer models served through Hugging Face free Inference API or Hugging Face TGI



In [8]:
from haystack import Pipeline
from haystack.components.builders.prompt_builder import PromptBuilder
from haystack.components.generators import HuggingFaceTGIGenerator
from haystack.components.embedders import SentenceTransformersTextEmbedder
from haystack.components.retrievers.in_memory import InMemoryEmbeddingRetriever
from haystack.utils import ComponentDevice

open_chat_prompt = """
GPT4 Correct User: You will be provided with a transcription of a recording with each sentence or group of sentences attributed to a Speaker by the word "Speaker" followed by a letter representing the person uttering that sentence. Answer the given question based on the given context.
If you think that given transcription is not enough to answer the question, say so.

Transcription:
{% for doc in documents %}
  {% if doc.meta["speaker"] %} Speaker {{doc.meta["speaker"]}}: {% endif %}{{doc.content}}
{% endfor %}
Question: {{ question }}
<|end_of_turn|>
GPT4 Correct Assistant:
"""

retriever = InMemoryEmbeddingRetriever(speaker_document_store)
text_embedder = SentenceTransformersTextEmbedder(device=ComponentDevice.from_str("cuda:0"))
answer_generator = HuggingFaceTGIGenerator("mistralai/Mistral-7B-Instruct-v0.2", generation_kwargs={"max_new_tokens":500})
prompt_builder = PromptBuilder(template=open_chat_prompt)

speaker_rag_pipe = Pipeline()
speaker_rag_pipe.add_component("text_embedder", text_embedder)
speaker_rag_pipe.add_component("retriever", retriever)
speaker_rag_pipe.add_component("prompt_builder", prompt_builder)
speaker_rag_pipe.add_component("llm", answer_generator)

speaker_rag_pipe.connect("text_embedder.embedding", "retriever.query_embedding")
speaker_rag_pipe.connect("retriever.documents", "prompt_builder.documents")
speaker_rag_pipe.connect("prompt_builder.prompt", "llm.prompt")

🚅 Components
  - text_embedder: SentenceTransformersTextEmbedder
  - retriever: InMemoryEmbeddingRetriever
  - prompt_builder: PromptBuilder
  - llm: HuggingFaceTGIGenerator
🛤️ Connections
  - text_embedder.embedding -> retriever.query_embedding (List[float])
  - retriever.documents -> prompt_builder.documents (List[Document])
  - prompt_builder.prompt -> llm.prompt (str)

## Test RAG with Speaker Labels

In [9]:
question = "What are each speakers' opinions on building in-house or using third parties?" # @param ["What are the two opposing opinions and how many people are on each side?", "What are each speakers' opinions on building in-house or using third parties?", "How many people are speaking in this recording?" ,"How many speakers and moderators are in this call?"]

result = speaker_rag_pipe.run({
    "prompt_builder":{"question": question},
    "text_embedder":{"text": question},
    "retriever":{"top_k": 10}
})
result["llm"]["replies"][0]

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

' Based on the context provided in the transcription, each speaker expresses a nuanced view on the decision to build in-house or use third parties. Speaker B emphasizes the importance of considering whether the potential project is core IP for their business and the value it brings. They suggest a mixed approach, using market standard solutions when appropriate and building in-house for core IP. Speaker A asks questions about the resources and constraints of the companies, suggesting that the decision may depend on factors such as budget and team capabilities. Speaker C emphasizes the importance of embedding AI capabilities into the business and using a mix of open source, partnerships, and in-house development. Overall, the speakers seem to agree that the decision is not a simple one and requires careful consideration of various factors.'